In [5]:
import numpy as np
import math
import pandas as pd

In [181]:
def initParams(a: float, b: float, n: int, diferSecond):
    """
    Иницилизирует начальные данные
    :param a: левая граница
    :param b: правая граница
    :param n: количество разбиений
    :param diferSecond: лямбда второй производной
    :return: Возврашает проинецилизорованые данные
    """
    left_matrix = np.zeros((n, n))
    center_matrix = np.zeros(n)
    right_matrix = np.zeros(n)
    lambda_array = np.zeros(n)
    u_array = np.zeros(n)
    h = (b - a) / n

    x_matrix = np.zeros(n)
    for i in range(n):
        x_matrix[i] = (a + i * h)
    left_matrix[0, 0] = 1
    left_matrix[0, 1] = 0
    left_matrix[- 1, -2] = 0
    left_matrix[- 1, -1] = 1
    right_matrix[0] = diferSecond(a)
    right_matrix[-1] = diferSecond(b)

    return left_matrix, center_matrix, right_matrix, u_array, h, x_matrix, lambda_array


In [189]:
def splainThird(x, m, h, n, value, fun):
    """

    :param x: матрица левая
    :param m: матрица решений
    :param h: шаг
    :param n: количество разбиений
    :param value: искомое значение
    :param fun: исходная функция
    :return: значение сплайна в точке
    """
    index = -100
    for i in range(n):
        if x[i] <= value <= x[i + 1]:
            index = i +1
            break
    print(x.shape)
    print(index)
    splain_value = (((x[index] - value) ** 3 - (h ** 2) * (x[index] - value)) * m[index - 1]) / (6 * h) + (
            ((value - x[index - 1]) ** 3 - (h ** 2) * (value - x[index - 1])) * m[index]) / (6 * h) + (
                           (x[index] - value) * fun(x[index - 1])) / h + ((value - x[index - 1]) * fun(x[index])) / h
    return splain_value


In [190]:
def calculateSplain(a: float, b: float, n: int,
                    fun, diferSecond):
    """
    Расчет массивов методом прогонки
    :param a: левая граница
    :param b: правая граница
    :param n: количество разбиений
    :param fun: лямбда функция искомой функции
    :param diferSecond: лямбда второй производной
    :return: Матрицу коэффициентов, матрицу решений, количество разбиений, шаг
    """
    left_matrix, center_matrix, right_matrix, u_array, h, x_matrix, lambda_array = initParams(a, b, n,
                                                                                              diferSecond)
    for i in range(1, n - 1):
        left_matrix[i, i - 1] = h / 6  # Присваеваем коэф с
        left_matrix[i, i] = 2 * h / 3  # коэф а
        left_matrix[i, i + 1] = h / 6  # коэф b

        right_matrix[i] = (fun(x_matrix[i + 1]) - fun(x_matrix[i])) / h - (fun(x_matrix[i]) - fun(x_matrix[i - 1])) / h

    answer = np.linalg.solve(left_matrix, right_matrix)

    return x_matrix, answer, n, h


In [191]:
deltaMax =[0]
count = [5]
spines = [0]

x_matrix, center_mas, n, h = calculateSplain(a=0.0, b=math.pi, n=5, fun=lambda x: math.sin(x),
                                             diferSecond=lambda x: -math.sin(x))
spine = splainThird(x_matrix, center_mas, h, 5, x_matrix[3] + h/2, fun=lambda x: math.sin(x))
spines[0] = spine
deltaMax[0] = abs(spine - math.sin(x_matrix[3] + h/2))
count.append(count[-1]*2)

for i in range(10):
    x_matrix, center_mas, n, h = calculateSplain(a=0.0, b=math.pi, n=count[-1], fun=lambda x: math.sin(x),
                                                 diferSecond=lambda x: -math.sin(x))
    spine = splainThird(x_matrix, center_mas, h, n, 1 - h/2, fun=lambda x: math.sin(x))
    spines.append(spine)
    deltaMax.append(abs(spine - math.sin( 1 - h/2)))
    count.append(count[-1]*2)
    
count.pop()

result = np.array([count, deltaMax]).transpose()
df = pd.DataFrame(result, columns=['count','deltaMax'])
df

(5,)
4
(10,)
3
(20,)
6
(40,)
13
(80,)
25
(160,)
51
(320,)
102
(640,)
204
(1280,)
407
(2560,)
815
(5120,)
1630


,count,deltaMax
0,5.0,1.133385e-02
1,10.0,1.358610e-05
2,20.0,2.359103e-07
3,40.0,4.280395e-08
4,80.0,8.495737e-11
5,160.0,3.114943e-10
6,320.0,1.724487e-11
7,640.0,6.074030e-13
8,1280.0,3.330669e-16
9,2560.0,3.763656e-14


In [173]:
import matplotlib.pyplot as plt

In [ ]:
%pylab inline
x = linspace(0, math.pi*2, n)
y = np.zeros(n)
for i in range(n):
    y[i] = splainThird(x_matrix, center_mas, h, n, x[i], fun=lambda x: math.sin(x))
plt.plot(x,y, label='line 1')
plt.show()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0
1
2
3
4
5
6
7
8
